development of functions for baseline droplet chemistry scripts

In [1]:
import ruamel.yaml
import pandas as pd
import os

In [2]:
notebook_wd = os.getcwd()

# imports

will be updating scripts here iteratively, so the functions in these scripts should reflect changes made to the functions in the space below

In [3]:
os.chdir(os.path.join(notebook_wd, '..'))
from src.d00_utils.conf_utils import *
from src.d00_utils.calc_utils import *

In [4]:
project_dir = get_project_directory()
cmpds, water = load_compounds()
cmpds

{'triethylene_glycol': {'name': 'PEG-3',
  'D_g': 5.95e-06,
  'mw': 0.1502,
  'rho': 1108.0,
  'c_inf': 0,
  'vp': 0.0668,
  'T_vp': 298,
  'dH': 78300.0},
 'hexaethylene_glycol': {'name': 'PEG-6',
  'D_g': 4.26e-06,
  'mw': 0.2823,
  'rho': 1180.0,
  'c_inf': 0,
  'vp': 3.05e-05,
  'T_vp': 298,
  'dH': 102100.0}}

In [5]:
params = load_parameters()
params

{'r_init': 5e-05,
 'X_h2o': 0.7,
 'composition': {'triethylene_glycol': 0.1, 'hexaethylene_glycol': 0.1}}

# initialization

In [6]:
from scipy.constants import pi

def calculate_volume_from_radius(r):
    """ Calculate volume from radius.

    Parameters
    ----------
    r : float
    radius of droplet in m.

    Returns
    -------
    V : float
    volume of droplet in m^3.
    """

    r = float(r)
    V = 4. / 3. * pi * r ** 3

    return V

In [7]:
v_init = calculate_volume_from_radius(params['r_init'])
v_init

5.235987755982989e-13

In [8]:
def normalize(unnormalized_array):
    """ Takes in an unnormalized array and normalizes it.

    Parameters
    ----------
    unnormalized_array : array
    array of some values that are unnormalized

    Returns
    -------
    normalized_array : array
    array of some values that are normalized
    """

    total = unnormalized_array.sum()
    normalized_array = np.true_divide(unnormalized_array, total)

    return normalized_array

In [9]:
comp = np.array(list(params['composition'].values()))
comp = normalize(comp)
print(comp)

[0.5 0.5]


In [10]:
def calculate_mole_fractions_from_molar_abundances(composition, x_water=0):
    
    if type(composition) is dict:
        composition = np.array(list(composition.values()))
    
    composition = normalize(composition)
    xs_cmpd = composition * (1 - x_water)
    
    return xs_cmpd

In [11]:
def calculate_moles_from_volume(V, compounds, water, xs_cmpd, x_water=0):
    """ Calculate volume from the mole fractions of a list of compounds in solution.

    Parameters
    ----------
    V : float
    Volume of solution in m^3.
    
    compounds : dict
    Dict of dicts for each component.

    composition : numpy.array or dict
    2D numpy array of molar amounts of non-aqueous material. First index is entry number
    (e.g., each timestep), second index is index of component in `components`.
    
    X_h2o : float
    mole fraction of water in solution.

    Returns
    -------
    ns_compound : numpy.array
    1D array of moles of compounds according to composition and droplet size.
    """

    xs = np.append(xs_cmpd, x_water)
    cmpds = { **compounds, **{'water' : water} }

    mw_avg = np.average([defs['mw'] for name, defs in cmpds.items()],
                        weights=xs)

    rho_avg = np.average([defs['rho'] for name, defs in cmpds.items()],
                         weights=xs)

    m_total = V * rho_avg
    n_total = m_total / mw_avg
    ns_cmpd = xs_cmpd * n_total

    return ns_cmpd

In [12]:
xs_cmpd = calculate_mole_fractions_from_molar_abundances(params['composition'], x_water=0)
ns_droplet = calculate_moles_from_volume(v_init, cmpds, water, xs_cmpd, x_water=0)
ns_droplet

array([1.38496416e-09, 1.38496416e-09])

# building of the evaporation model

In [13]:
def calculate_volume_from_moles(compounds, ns):
 
    mw_avg = np.average([defs['mw'] for name, defs in compounds.items()],
                        weights=ns)

    rho_avg = np.average([defs['rho'] for name, defs in compounds.items()],
                         weights=ns)
    
    n_total = ns.sum()
    V = n_total * mw_avg / rho_avg

    return V

In [15]:
import numpy as np
from scipy.constants import k, pi

from src.d00_utils.calc_utils import convert_moles_to_volume, convert_volume_to_radius
from src.d00_utils.misc_utils import normalize


def dn_gas_particle_partitioning(ns_cmpd, c_infs, vps, D_gs, T, compounds, water, x_water=0):
    '''Construct differential equations to describe change in n.

    Parameters:
    -----------
    t : float
    Time
    y: ndarray
    1D-array of number of molecules of each component.
    Ma : ndarray
    1D-array of molecular weights, kg molec^-1.
    rho : ndarray
    1D-array of number of densities, kg m^-3.
    cinf : ndarray
    1D-array of concentration of substance at infinite distance, molec m^-3.
    po : ndarray
    1D-array of pure compound vapor pressures, Pa.
    Dg : ndarray
    1D-array of gas-phase diffusivities, m^2 s^-1.
    T : float
    Temperature, K.
    has_water : boolean
    Include water in calculation.
    xh2o : float
    Fixed water mole fraction. Only used if has_water is True.

    Outputs
    -------
    dn : ndarray
    1D-array of dn for all components.
    '''

    n_water = x_water * ns_cmpd.sum() / (1 - x_water)
    
    # add water back to compounds and ns for radius calculation
    ns = np.append(ns_cmpd, n_water)
    cmpds = { **compounds, **{'water' : water} }
    
    V = convert_moles_to_volume(cmpds, ns)
    r = convert_volume_to_radius(V)
    
    xs = calculate_mole_fractions_from_molar_abundances(ns_cmpd, x_water=x_water)
    c_sats = xs * vps / T  # assume ideal mixing to calculate saturation concentration at surface
    
    # maxwellian flux
    dns = 4 * pi * r * (D_gs * (c_infs - c_sats))

    return dns

In [17]:
c_infs = [compound_defs['c_inf'] for compound_name, compound_defs in cmpds.items()]
D_gs = [compound_defs['D_g'] for compound_name, compound_defs in cmpds.items()]
vps = [compound_defs['vp'] for compound_name, compound_defs in cmpds.items()]
T = 298

dns_droplet = dn_gas_particle_partitioning(ns_droplet, c_infs, vps, D_gs, T, cmpds, water, x_water=0)
dns_droplet

array([-4.19012556e-13, -1.36975548e-16])

In [18]:
x_water = 0
dict_test = {'c_infs' : c_infs, 'vps' : vps, 'D_gs' : D_gs, 'T' : T, 'cmpds' : cmpds, 'water' : water, 'x_water' : x_water}

In [51]:
import numpy as np
from scipy.constants import pi

from src.d00_utils.calc_utils import (convert_moles_to_volume, convert_volume_to_radius,
                                      convert_molar_abundances_to_mole_fractions, calculate_vp_from_reference)
from src.d00_utils.misc_utils import unpack_dictionary


def dn_gas_particle_partitioning(t, n_cmpds, gas_particle_partitioning_dict):
    """Construct differential equations to describe change in n.

    Parameters:
    -----------
    t : int
    time step in seconds.
    ns_cmpd : ndarray
    1D-array of moles of compounds.
    c_infs: ndarray
    1D-array of gas-phase background concentrations of compounds, mol m^-3.
    vps : ndarray
    1D-array of saturation vapor pressures at T, Pa.
    D_gs : ndarray
    1D-array of gas-phase diffusivities, m^2 s^-1.
    T : float
    temperature, K.
    compounds : dict(dict)
    dictionary of compounds and their definitions.
    water : dict
    dictionary of water definitions.
    x_water : float
    water mole fraction.

    Outputs
    -------
    dn : ndarray
    1D-array of dn for all compounds (excluding water).
    """

    def unpack(c_infs, D_gs, vps, T, compounds, water, x_water):
        return c_infs, D_gs, vps, T, compounds, water, x_water
    
    c_infs, D_gs, vps, T, compounds, water, x_water = unpack(**gas_particle_partitioning_dict)
    
    try:
        x_water, compounds, water, vps, T, D_gs, c_infs
    except NameError:
        print('evaporate_params dictionary is missing variables')

    n_water = x_water * n_cmpds.sum() / (1 - x_water)

    # add water back to compounds and ns for radius calculation
    ns = np.append(n_cmpds, n_water)
    cmpds = {**compounds, **{'water': water}}

    V = convert_moles_to_volume(compounds=cmpds,
                                ns=ns)
    r = convert_volume_to_radius(V=V)

    xs = convert_molar_abundances_to_mole_fractions(composition=n_cmpds,
                                                    x_water=x_water)
    c_sats = xs * vps / T  # assume ideal mixing to calculate saturation concentration at surface

    # maxwellian flux
    dns = 4 * pi * r * (D_gs * (c_infs - c_sats))

    return dns


def pack_evaporate_params(compounds, water, T, x_water=0):
    '''Calculate evaporation of multicomponent particle.

    Parameters
    ----------
    compounds : list
    List of compounds of particles, each represented as a dict of parameters.

    ninit : numpy.ndarray
    1D array of starting number of molecules of each compound in `compounds`.

    T : float
    Temperature of evaporation, K.

    num : int
    Total number of integrated time points, including t=0.

    dt : float
    Integration timestep, s.

    has_water : boolean (optional, default False)
    Toggle whether the presence of water is considered in calculating
    evaporation (separate from list of `compounds`).

    xh2o : float (optional, default None)
    Fixed mole fraction of water to include in particle. Only considered if
    `has_water` is True.

    Returns
    -------
    output : numpy.ndarray
    2D array of results from integration: number of molecules of each compound
    remaining at each timestep. First index is along `num` timesteps and second
    index is along `len(compounds)` compounds.
    '''

    # extract data from compounds and build data arrays
    c_infs = np.array([defs['c_inf'] for name, defs in compounds.items()])
    D_gs = np.array([defs['D_g'] for name, defs in compounds.items()])
    
    vp_refs = np.array([defs['vp'] for name, defs in compounds.items()])
    dHs = np.array([defs['dH'] for name, defs in compounds.items()])
    T_refs = np.array([defs['T_vp'] for name, defs in compounds.items()])
    
    vps = np.empty(len(vp_refs))
    for tick in range(len(vps)):
        vps[tick] = calculate_vp_from_reference(vp_ref=vp_refs[tick],
                                                dH=dHs[tick],
                                                T_ref=T_refs[tick],
                                                T_desired=T)

    gas_particle_partitioning_dict = {'c_infs': c_infs, 'D_gs': D_gs, 'vps': vps,
                                      'T': T, 'compounds': compounds, 'water': water, 'x_water': x_water}

    return gas_particle_partitioning_dict


In [50]:
gp_dict = pack_evaporate_params(cmpds, water, T=298, x_water=0.3)
t=1
dns_test = dn_gas_particle_partitioning(t, ns_droplet, gp_dict)
dns_test

array([-3.00670095e-13, -9.82893008e-17])

In [52]:
import numpy as np
from scipy.integrate import ode

def perform_dn(function, function_params_dict, n_inits, step_count, step=1):
    """"""

    output = np.empty((int(step_count), len(n_inits)))
    output[0, :] = n_inits

    r = ode(function)
    r.set_integrator('lsoda', with_jacobian=False,)
    r.set_initial_value(n_inits, t=0)
    r.set_f_params(function_params_dict)

    entry = 0
    while r.successful() and entry < step_count - 1:
        entry = int(round(r.t / step)) + 1
        next_step = r.integrate(r.t + step)
        output[entry, :] = next_step

    return output

In [56]:
perform_dn(dn_gas_particle_partitioning, gp_dict, ns_droplet, 100000, step=100)

array([[1.38496416e-09, 1.38496416e-09],
       [1.35510007e-09, 1.38495429e-09],
       [1.32564182e-09, 1.38494434e-09],
       ...,
       [2.88449753e-15, 1.15445073e-10],
       [2.88454692e-15, 1.15437792e-10],
       [2.88459629e-15, 1.15430511e-10]])